# Business Analytics - Assignment 1 - Question 2
#### **Student Name:** Koorosh Shakoori

# Computer Assembly Plan
This task falls into the Multiperiod Production Models Category.

In [2]:
from pulp import *

## Defining the decision variables
To define the decision variables define a weeks list to iterate over for defining Regular, Overtime and, inventory cost variables.

In [3]:
weeks = ['Week1', 'Week2', 'Week3', 'Week4', 'Week5', 'Week6']
Regular_Cost = 190
Overtime_Cost = 260
Inventory_Cost = 10

Regular = LpVariable.dicts('Regular_production', weeks, 0)
Overtime = LpVariable.dicts('Overtime_Production', weeks, 0)
Inventory = LpVariable.dicts('Inventory', weeks, 0)

## Defining the Objective Function
Given the cost of each operation, the problem asks for minimum cost while meeting all the conditions.
Therefore the objective function is defined as below:

In [4]:
#initializing the LpProblem object from pulp with minimize functionality
prob = LpProblem('Computer_Assembly_Plan', LpMinimize)
prob += lpSum([Regular[week]*Regular_Cost + Overtime[week]*Overtime_Cost + Inventory[week]*Inventory_Cost for week in weeks])

## Defining Constraints
Since this is a multiperiod production model, we need to define supply-demand balance equations as constraints.
On top of that we will have to limit the regular and overtime productions to their respective amount.

In [5]:
#we first put the limits on production rates
for week in weeks:
    prob += Regular[week] <= 160, f'{week} regular production limit'
    prob += Overtime[week] <= 50, f'{week} overtime production limit'

#Balance Equations
prob += Regular['Week1'] + Overtime['Week1'] - Inventory['Week1'] == 105, 'Week1_demand'
prob += Regular['Week2'] + Overtime['Week2'] + Inventory['Week1'] - Inventory['Week2'] == 170, 'Week2_demand'
prob += Regular['Week3'] + Overtime['Week3'] + Inventory['Week2'] - Inventory['Week3'] == 230, 'Week3_demand'
prob += Regular['Week4'] + Overtime['Week4'] + Inventory['Week3'] - Inventory['Week4'] == 180, 'Week4_demand'
prob += Regular['Week5'] + Overtime['Week5'] + Inventory['Week4'] - Inventory['Week5'] == 150, 'Week5_demand'
prob += Regular['Week6'] + Overtime['Week6'] + Inventory['Week5'] == 250, 'Week6_demand'

In [6]:
#Here we get an overall report on all sides of the problem
prob

Computer_Assembly_Plan:
MINIMIZE
10*Inventory_Week1 + 10*Inventory_Week2 + 10*Inventory_Week3 + 10*Inventory_Week4 + 10*Inventory_Week5 + 10*Inventory_Week6 + 260*Overtime_Production_Week1 + 260*Overtime_Production_Week2 + 260*Overtime_Production_Week3 + 260*Overtime_Production_Week4 + 260*Overtime_Production_Week5 + 260*Overtime_Production_Week6 + 190*Regular_production_Week1 + 190*Regular_production_Week2 + 190*Regular_production_Week3 + 190*Regular_production_Week4 + 190*Regular_production_Week5 + 190*Regular_production_Week6 + 0
SUBJECT TO
Week1_regular_production_limit: Regular_production_Week1 <= 160

Week1_overtime_production_limit: Overtime_Production_Week1 <= 50

Week2_regular_production_limit: Regular_production_Week2 <= 160

Week2_overtime_production_limit: Overtime_Production_Week2 <= 50

Week3_regular_production_limit: Regular_production_Week3 <= 160

Week3_overtime_production_limit: Overtime_Production_Week3 <= 50

Week4_regular_production_limit: Regular_production_Week4 

In [7]:
prob.solve(PULP_CBC_CMD(msg=0))

1

In [8]:
#The optimal solution is as explained below.
#The overal cost is mention at the end.
for var in prob.variables():
    print(f'{var.name} = {var.varValue}')
print(f'Overall cost of production = {prob.objective.value()}')

Inventory_Week1 = 55.0
Inventory_Week2 = 45.0
Inventory_Week3 = 0.0
Inventory_Week4 = 0.0
Inventory_Week5 = 40.0
Inventory_Week6 = 0.0
Overtime_Production_Week1 = 0.0
Overtime_Production_Week2 = 0.0
Overtime_Production_Week3 = 25.0
Overtime_Production_Week4 = 20.0
Overtime_Production_Week5 = 30.0
Overtime_Production_Week6 = 50.0
Regular_production_Week1 = 160.0
Regular_production_Week2 = 160.0
Regular_production_Week3 = 160.0
Regular_production_Week4 = 160.0
Regular_production_Week5 = 160.0
Regular_production_Week6 = 160.0
Overall cost of production = 216300.0


In [13]:
import pandas as pd
#Report Sensitivity Analysis
print("\nSensitivity Analysis")
Cons_Sensitivity_Report=[{'Constraint_Name':name,'Slack':c.slack,"Shadow_Price":c.pi}
                         for name, c in prob.constraints.items()]
print(pd.DataFrame(Cons_Sensitivity_Report))


Sensitivity Analysis
                    Constraint_Name  Slack  Shadow_Price
0    Week1_regular_production_limit   -0.0         -50.0
1   Week1_overtime_production_limit   50.0           0.0
2    Week2_regular_production_limit   -0.0         -60.0
3   Week2_overtime_production_limit   50.0           0.0
4    Week3_regular_production_limit   -0.0         -70.0
5   Week3_overtime_production_limit   25.0           0.0
6    Week4_regular_production_limit   -0.0         -70.0
7   Week4_overtime_production_limit   30.0           0.0
8    Week5_regular_production_limit   -0.0         -70.0
9   Week5_overtime_production_limit   20.0           0.0
10   Week6_regular_production_limit   -0.0         -80.0
11  Week6_overtime_production_limit   -0.0         -10.0
12                     Week1_demand   -0.0         240.0
13                     Week2_demand   -0.0         250.0
14                     Week3_demand   -0.0         260.0
15                     Week4_demand   -0.0         260.0
16       

## Answer to question B
**Week 6.** According to the shadow price chart, increasing the regular price capacity in week 6 will be the most impactful in lowering the cost, with each capacity reducing the overal cost by 80 dollars. This is due to lack of inventory cost in the last week. 